<a href="https://colab.research.google.com/github/MathMayhem/Binary-Neural-Networks/blob/main/Binary_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import copy

In [ ]:
for i in range(2000):
  if i % 100 == 0:
    print(i)
  update(model, binarized_model, x, y, loss_fn)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


In [ ]:
binarize(model, binarized_model)
binarized_model(x)

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.50000036],
       [0.50000036],
       [0.50000036],
       [0.50000036]], dtype=float32)>

In [ ]:
binarized_model.weights

[<Variable path=dense_20/kernel, shape=(2, 4), dtype=float32, value=[[-1. -1. -1. -1.]
  [ 1. -1. -1. -1.]]>,
 <Variable path=dense_20/bias, shape=(4,), dtype=float32, value=[-0.88136107 -0.81576234 -0.8914648  -0.31798455]>,
 <Variable path=dense_21/kernel, shape=(4, 4), dtype=float32, value=[[-1. -1. -1. -1.]
  [-1. -1. -1. -1.]
  [-1. -1.  1. -1.]
  [-1.  1. -1.  1.]]>,
 <Variable path=dense_21/bias, shape=(4,), dtype=float32, value=[-0.2828127  -0.75917333 -0.5175005  -0.78174233]>,
 <Variable path=dense_22/kernel, shape=(4, 4), dtype=float32, value=[[-1.  1. -1.  1.]
  [-1. -1. -1. -1.]
  [ 1. -1.  1. -1.]
  [-1. -1.  1. -1.]]>,
 <Variable path=dense_22/bias, shape=(4,), dtype=float32, value=[-0.38474303 -0.5443923  -0.3658104  -0.3034463 ]>,
 <Variable path=dense_23/kernel, shape=(4, 4), dtype=float32, value=[[-1.  1. -1. -1.]
  [-1. -1. -1.  1.]
  [-1.  1. -1. -1.]
  [-1. -1.  1. -1.]]>,
 <Variable path=dense_23/bias, shape=(4,), dtype=float32, value=[-0.19593303 -0.3029064  -0.

In [ ]:
# Stochastic version of Binary Connect
def binarize(model, binarized_model):
  binarized_weights = [tensor if tensor.name == 'bias' else vec_f(tensor) for tensor in model.weights]
  binarized_model.set_weights(binarized_weights)

def backward(model, binarized_model, inputs, targets, loss_fn):
  trainable_variables = binarized_model.trainable_variables
  total_gradients = [np.zeros(tensor.shape) for tensor in trainable_variables]

  for input, target in zip(inputs, targets):
    binarize(model, binarized_model)

    with tf.GradientTape() as tape:
      predictions = binarized_model(np.array([input]))[0]
      loss = loss_fn(target, predictions)

    gradients = tape.gradient(loss, trainable_variables)

    for i in range(len(total_gradients)):
      total_gradients[i] += gradients[i]

  return total_gradients

def update(model, binarized_model, inputs, targets, loss_fn):
  gradients = backward(model, binarized_model, inputs, targets, loss_fn)
  trainable_variables = model.trainable_variables
  model.optimizer.apply_gradients(zip(gradients, trainable_variables))
  model.set_weights([tensor if tensor.name == 'bias' else vec_g(tensor) for tensor in model.weights])

In [ ]:
def f(x):
  clipped_x = max(0.0, min(1.0, (x + 1.0)/2.0))
  if np.random.rand() < clipped_x:
    return 1.0
  else:
    return -1.0

vec_f = np.vectorize(f)

def g(x):
  return max(-1.0, min(1.0, x))

vec_g = np.vectorize(g)

In [ ]:
loss_fn = tf.keras.losses.MSE
architecture = [
  layers.Input((2,)),
  layers.Dense(4, activation = 'relu'),
  layers.Dense(4, activation = 'relu'),
  layers.Dense(4, activation = 'relu'),
  layers.Dense(4, activation = 'relu'),
  layers.Dense(1)
]
model = models.Sequential(architecture)
binarized_model = models.Sequential(copy.deepcopy(architecture))

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01)

model.compile(optimizer, loss_fn)
binarized_model.compile(optimizer)

In [ ]:
x = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
y = np.array([[0.0], [1.0], [1.0], [0.0]])

In [ ]:
model(x)

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[ 0.        ],
       [-0.05585981],
       [-0.02336417],
       [ 0.        ]], dtype=float32)>

In [ ]:
model.fit(x, y, epochs = 2000, verbose = 2)

Epoch 1/2000
1/1 - 1s - 609ms/step - loss: 0.3696
Epoch 2/2000
1/1 - 0s - 34ms/step - loss: 0.3418
Epoch 3/2000
1/1 - 0s - 36ms/step - loss: 0.3223
Epoch 4/2000
1/1 - 0s - 57ms/step - loss: 0.3081
Epoch 5/2000
1/1 - 0s - 31ms/step - loss: 0.2980
Epoch 6/2000
1/1 - 0s - 31ms/step - loss: 0.2903
Epoch 7/2000
1/1 - 0s - 32ms/step - loss: 0.2846
Epoch 8/2000
1/1 - 0s - 33ms/step - loss: 0.2803
Epoch 9/2000
1/1 - 0s - 59ms/step - loss: 0.2767
Epoch 10/2000
1/1 - 0s - 34ms/step - loss: 0.2737
Epoch 11/2000
1/1 - 0s - 59ms/step - loss: 0.2711
Epoch 12/2000
1/1 - 0s - 32ms/step - loss: 0.2689
Epoch 13/2000
1/1 - 0s - 60ms/step - loss: 0.2669
Epoch 14/2000
1/1 - 0s - 59ms/step - loss: 0.2650
Epoch 15/2000
1/1 - 0s - 34ms/step - loss: 0.2632
Epoch 16/2000
1/1 - 0s - 33ms/step - loss: 0.2615
Epoch 17/2000
1/1 - 0s - 33ms/step - loss: 0.2599
Epoch 18/2000
1/1 - 0s - 32ms/step - loss: 0.2583
Epoch 19/2000
1/1 - 0s - 59ms/step - loss: 0.2567
Epoch 20/2000
1/1 - 0s - 59ms/step - loss: 0.2552
Epoch 21